In [10]:
import psycopg2
import psycopg2.extras
import os

hostname = '' # <--- enter hostname
database = '' # <--- enter database name
username = '' # <--- enter username
pwd = '' # <--- enter password
port_id =  # <--- enter port id
table = '' # <--- enter table name
conn = None
path='' # <--- enter diretory path of file(s)

def create_table(table_name):
    return f'''CREATE TABLE IF NOT EXISTS {table_name} (
            key VARCHAR(100) PRIMARY KEY NOT NULL,
            gene_symbol VARCHAR(40) NOT NULL,
            PasID VARCHAR(40) NOT NULL,
            type_main VARCHAR(2) NOT NULL,
            PSE NUMERIC(4,1) CHECK(PSE >= 0), 
            AvgRPM NUMERIC(10,2) CHECK(AvgRPM >=0), 
            mm10_pAid VARCHAR(40),
            NumRefSeq NUMERIC(10) CHECK(NumRefSeq >=0), 
            NumLRENCODE NUMERIC(10) CHECK(NumLRENCODE >=0), 
            NumLRGETx NUMERIC(10) CHECK(NumLRGETx >=0),
            polyAID NUMERIC(5,4) CHECK(polyAID >=0), 
            polyAStrength NUMERIC(10,4),
            SVM NUMERIC(5,4) CHECK(SVM >=0))'''    

def insert_data(file, table_name):
    return f"""COPY {table_name}(key, 
                                gene_symbol, 
                                PasID, 
                                type_main, 
                                PSE, 
                                AvgRPM, 
                                mm10_pAid, 
                                NumRefSeq, 
                                NumLRENCODE, 
                                NumLRGETx, 
                                polyAID, 
                                polyAStrength, 
                                SVM) FROM '{file}' DELIMITER ',' CSV HEADER"""


try:
    with psycopg2.connect(
        host = hostname,
        dbname = database,
        password = pwd,
        port = port_id) as conn:
    
        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

            cur.execute(f'DROP TABLE IF EXISTS {table}')

            #### create table
            create = create_table(table)
            cur.execute(create)
            
            #### insert data (csv format) into table 
            for filename in os.listdir(path):
                file = path + '/'  + filename
                insert = insert_data(file, table)
                cur.execute(insert)
            
            conn.commit()

except Exception as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
